In [1]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
import numpy as np
import pprint
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

# Overview
This notebook was useed to test binance python api, for data downloading and simple processing it. 

In [2]:
### API
bitmex_api_key = '[REDACTED]'    #Enter your own API-key here
bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = '[REDACTED]'    #Enter your own API-key here
binance_api_secret = '[REDACTED]' #Enter your own API-secret here

In [1]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [62]:
info = binance_client.get_exchange_info()

In [68]:
for symbol in info['symbols']:
    if symbol['symbol'] == 'BTCUSDT':
        pprint.pp(symbol)

{'symbol': 'BTCUSDT',
 'status': 'TRADING',
 'baseAsset': 'BTC',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
                'LIMIT_MAKER',
                'MARKET',
                'STOP_LOSS_LIMIT',
                'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
              'minPrice': '0.01000000',
              'maxPrice': '1000000.00000000',
              'tickSize': '0.01000000'},
             {'filterType': 'PERCENT_PRICE',
              'multiplierUp': '5',
              'multiplierDown': '0.2',
              'avgPriceMins': 5},
             {'filterType': 'LOT_SIZE',
              'minQty': '0.00000100',
              'maxQty': '9000.00000000',
              'stepSize': '0.

In [4]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = 'data_raw/crypto/%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = 'data_raw/crypto/%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [6]:
data = get_all_binance('BTCUSDT', '5m', save=False)
data

All caught up..!


,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48000000,4280.56000000,4261.48000000,4261.48000000,2.18906100,1502942699999,9333.62096220,9,0.48906100,2089.10496220,7960.37529526
2017-08-17 04:05:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502942999999,0.00000000,0,0.00000000,0.00000000,7959.62629202
2017-08-17 04:10:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502943299999,0.00000000,0,0.00000000,0.00000000,7958.41741512
2017-08-17 04:15:00,4261.48000000,4264.88000000,4261.48000000,4261.48000000,0.48466600,1502943599999,2065.44782868,9,0.08545500,364.21813640,7958.34896534
2017-08-17 04:20:00,4264.88000000,4266.29000000,4264.88000000,4266.29000000,2.32857000,1502943899999,9931.16112417,11,1.54649100,6595.68803865,7953.96934947
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-04 18:15:00,36990.00000000,37075.92000000,36986.93000000,37050.55000000,124.84239000,1622830799999,4623124.06147883,3449,60.46021000,2238615.45805476,0
2021-06-04 18:20:00,37051.56000000,37109.03000000,37048.84000000,37091.75000000,116.99201500,1622831099999,4338592.56315653,3024,52.12492600,1932902.38924766,0
2021-06-04 18:25:00,37091.75000000,37159.79000000,37084.00000000,37124.16000000,185.28952300,1622831399999,6876867.68713567,3519,97.75135500,3627987.20565784,0


In [7]:
data1m = get_all_binance('BTCUSDT', '1m', save=False)
data1m

All caught up..!


,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-04 18:53:00,37150.65000000,37164.08000000,37090.73000000,37113.84000000,47.04432100,1622832839999,1746922.98558496,901,22.39955100,831984.83438102,0
2021-06-04 18:54:00,37113.83000000,37130.97000000,37106.57000000,37108.99000000,18.43840600,1622832899999,684427.22859709,692,8.11491400,301232.67164187,0
2021-06-04 18:55:00,37108.99000000,37150.00000000,37106.20000000,37131.70000000,24.85349200,1622832959999,922865.32927491,779,11.79525300,438008.36654989,0


In [8]:
data1m

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-04 18:53:00,37150.65000000,37164.08000000,37090.73000000,37113.84000000,47.04432100,1622832839999,1746922.98558496,901,22.39955100,831984.83438102,0
2021-06-04 18:54:00,37113.83000000,37130.97000000,37106.57000000,37108.99000000,18.43840600,1622832899999,684427.22859709,692,8.11491400,301232.67164187,0
2021-06-04 18:55:00,37108.99000000,37150.00000000,37106.20000000,37131.70000000,24.85349200,1622832959999,922865.32927491,779,11.79525300,438008.36654989,0


In [15]:
data1m.index

DatetimeIndex(['2017-08-17 04:00:00', '2017-08-17 04:01:00',
               '2017-08-17 04:02:00', '2017-08-17 04:03:00',
               '2017-08-17 04:04:00', '2017-08-17 04:05:00',
               '2017-08-17 04:06:00', '2017-08-17 04:07:00',
               '2017-08-17 04:08:00', '2017-08-17 04:09:00',
               ...
               '2021-06-04 18:48:00', '2021-06-04 18:49:00',
               '2021-06-04 18:50:00', '2021-06-04 18:51:00',
               '2021-06-04 18:52:00', '2021-06-04 18:53:00',
               '2021-06-04 18:54:00', '2021-06-04 18:55:00',
               '2021-06-04 18:56:00', '2021-06-04 18:57:00'],
              dtype='datetime64[ns]', name='timestamp', length=1990087, freq=None)

In [17]:
dates = data1m.index.strftime('%Y-%m-%d')

(array(['2017-08-17', '2017-08-18', '2017-08-19', ..., '2021-06-02',
        '2021-06-03', '2021-06-04'], dtype=object),
 array([      0,    1200,    2640, ..., 1986069, 1987509, 1988949]))

In [18]:
indices = np.unique(dates, return_index=True)[1]
dates[indices]

Index(['2017-08-17', '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21',
       '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25', '2017-08-26',
       ...
       '2021-05-26', '2021-05-27', '2021-05-28', '2021-05-29', '2021-05-30',
       '2021-05-31', '2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04'],
      dtype='object', name='timestamp', length=1388)

In [25]:
data1m = data1m.drop(columns='datetime64')

In [32]:
features = data1m[['open', 'high', 'low', 'close', 'volume']]
features

,open,high,low,close,volume
timestamp,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600
...,...,...,...,...,...
2021-06-04 18:53:00,37150.65000000,37164.08000000,37090.73000000,37113.84000000,47.04432100
2021-06-04 18:54:00,37113.83000000,37130.97000000,37106.57000000,37108.99000000,18.43840600
2021-06-04 18:55:00,37108.99000000,37150.00000000,37106.20000000,37131.70000000,24.85349200


In [35]:
features = features.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})

In [43]:
features['time_indicator'] = (features.index.hour * 3600 + features.index.minute * 60) / (24 * 3600)
features

,Open,High,Low,Close,Volume,time_indicator
timestamp,,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300,0.166667
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,0.167361
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400,0.168056
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800,0.168750
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600,0.169444
...,...,...,...,...,...,...
2021-06-04 18:53:00,37150.65000000,37164.08000000,37090.73000000,37113.84000000,47.04432100,0.786806
2021-06-04 18:54:00,37113.83000000,37130.97000000,37106.57000000,37108.99000000,18.43840600,0.787500
2021-06-04 18:55:00,37108.99000000,37150.00000000,37106.20000000,37131.70000000,24.85349200,0.788194


In [57]:
features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1990087 entries, 2017-08-17 04:00:00 to 2021-06-04 18:57:00
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Open            float64
 1   High            float64
 2   Low             float64
 3   Close           float64
 4   Volume          float64
 5   time_indicator  float64
dtypes: float64(6)
memory usage: 106.3 MB


In [56]:
features['Open'] = pd.to_numeric(features['Open'])
features['High'] = pd.to_numeric(features['High'])
features['Low'] = pd.to_numeric(features['Low'])
features['Close'] = pd.to_numeric(features['Close'])
features['Volume'] = pd.to_numeric(features['Volume'])

In [59]:
features['avg_price_in_interval'] = (features['High'] + features['Low'])/2
features

,Open,High,Low,Close,Volume,time_indicator,avg_price_in_interval
timestamp,,,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,0.166667,4261.480
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,0.167361,4261.480
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,0.168056,4280.560
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,0.168750,4261.480
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,0.169444,4261.480
...,...,...,...,...,...,...,...
2021-06-04 18:53:00,37150.65,37164.08,37090.73,37113.84,47.044321,0.786806,37127.405
2021-06-04 18:54:00,37113.83,37130.97,37106.57,37108.99,18.438406,0.787500,37118.770
2021-06-04 18:55:00,37108.99,37150.00,37106.20,37131.70,24.853492,0.788194,37128.100


In [60]:
resampled = features.resample("D")
for name, group in resampled:
    filename = f"{name.strftime('%Y-%m-%d')}.csv"
    filepath = os.path.join('data_raw', 'crypto', 'daily', filename)
    group.to_csv(filepath)
    print("Group: ", filename, "saved.")

Group:  2017-08-17.csv saved.
Group:  2017-08-18.csv saved.
Group:  2017-08-19.csv saved.
Group:  2017-08-20.csv saved.
Group:  2017-08-21.csv saved.
Group:  2017-08-22.csv saved.
Group:  2017-08-23.csv saved.
Group:  2017-08-24.csv saved.
Group:  2017-08-25.csv saved.
Group:  2017-08-26.csv saved.
Group:  2017-08-27.csv saved.
Group:  2017-08-28.csv saved.
Group:  2017-08-29.csv saved.
Group:  2017-08-30.csv saved.
Group:  2017-08-31.csv saved.
Group:  2017-09-01.csv saved.
Group:  2017-09-02.csv saved.
Group:  2017-09-03.csv saved.
Group:  2017-09-04.csv saved.
Group:  2017-09-05.csv saved.
Group:  2017-09-06.csv saved.
Group:  2017-09-07.csv saved.
Group:  2017-09-08.csv saved.
Group:  2017-09-09.csv saved.
Group:  2017-09-10.csv saved.
Group:  2017-09-11.csv saved.
Group:  2017-09-12.csv saved.
Group:  2017-09-13.csv saved.
Group:  2017-09-14.csv saved.
Group:  2017-09-15.csv saved.
Group:  2017-09-16.csv saved.
Group:  2017-09-17.csv saved.
Group:  2017-09-18.csv saved.
Group:  20

Group:  2018-05-30.csv saved.
Group:  2018-05-31.csv saved.
Group:  2018-06-01.csv saved.
Group:  2018-06-02.csv saved.
Group:  2018-06-03.csv saved.
Group:  2018-06-04.csv saved.
Group:  2018-06-05.csv saved.
Group:  2018-06-06.csv saved.
Group:  2018-06-07.csv saved.
Group:  2018-06-08.csv saved.
Group:  2018-06-09.csv saved.
Group:  2018-06-10.csv saved.
Group:  2018-06-11.csv saved.
Group:  2018-06-12.csv saved.
Group:  2018-06-13.csv saved.
Group:  2018-06-14.csv saved.
Group:  2018-06-15.csv saved.
Group:  2018-06-16.csv saved.
Group:  2018-06-17.csv saved.
Group:  2018-06-18.csv saved.
Group:  2018-06-19.csv saved.
Group:  2018-06-20.csv saved.
Group:  2018-06-21.csv saved.
Group:  2018-06-22.csv saved.
Group:  2018-06-23.csv saved.
Group:  2018-06-24.csv saved.
Group:  2018-06-25.csv saved.
Group:  2018-06-26.csv saved.
Group:  2018-06-27.csv saved.
Group:  2018-06-28.csv saved.
Group:  2018-06-29.csv saved.
Group:  2018-06-30.csv saved.
Group:  2018-07-01.csv saved.
Group:  20

Group:  2019-03-09.csv saved.
Group:  2019-03-10.csv saved.
Group:  2019-03-11.csv saved.
Group:  2019-03-12.csv saved.
Group:  2019-03-13.csv saved.
Group:  2019-03-14.csv saved.
Group:  2019-03-15.csv saved.
Group:  2019-03-16.csv saved.
Group:  2019-03-17.csv saved.
Group:  2019-03-18.csv saved.
Group:  2019-03-19.csv saved.
Group:  2019-03-20.csv saved.
Group:  2019-03-21.csv saved.
Group:  2019-03-22.csv saved.
Group:  2019-03-23.csv saved.
Group:  2019-03-24.csv saved.
Group:  2019-03-25.csv saved.
Group:  2019-03-26.csv saved.
Group:  2019-03-27.csv saved.
Group:  2019-03-28.csv saved.
Group:  2019-03-29.csv saved.
Group:  2019-03-30.csv saved.
Group:  2019-03-31.csv saved.
Group:  2019-04-01.csv saved.
Group:  2019-04-02.csv saved.
Group:  2019-04-03.csv saved.
Group:  2019-04-04.csv saved.
Group:  2019-04-05.csv saved.
Group:  2019-04-06.csv saved.
Group:  2019-04-07.csv saved.
Group:  2019-04-08.csv saved.
Group:  2019-04-09.csv saved.
Group:  2019-04-10.csv saved.
Group:  20

Group:  2019-12-16.csv saved.
Group:  2019-12-17.csv saved.
Group:  2019-12-18.csv saved.
Group:  2019-12-19.csv saved.
Group:  2019-12-20.csv saved.
Group:  2019-12-21.csv saved.
Group:  2019-12-22.csv saved.
Group:  2019-12-23.csv saved.
Group:  2019-12-24.csv saved.
Group:  2019-12-25.csv saved.
Group:  2019-12-26.csv saved.
Group:  2019-12-27.csv saved.
Group:  2019-12-28.csv saved.
Group:  2019-12-29.csv saved.
Group:  2019-12-30.csv saved.
Group:  2019-12-31.csv saved.
Group:  2020-01-01.csv saved.
Group:  2020-01-02.csv saved.
Group:  2020-01-03.csv saved.
Group:  2020-01-04.csv saved.
Group:  2020-01-05.csv saved.
Group:  2020-01-06.csv saved.
Group:  2020-01-07.csv saved.
Group:  2020-01-08.csv saved.
Group:  2020-01-09.csv saved.
Group:  2020-01-10.csv saved.
Group:  2020-01-11.csv saved.
Group:  2020-01-12.csv saved.
Group:  2020-01-13.csv saved.
Group:  2020-01-14.csv saved.
Group:  2020-01-15.csv saved.
Group:  2020-01-16.csv saved.
Group:  2020-01-17.csv saved.
Group:  20

Group:  2020-09-21.csv saved.
Group:  2020-09-22.csv saved.
Group:  2020-09-23.csv saved.
Group:  2020-09-24.csv saved.
Group:  2020-09-25.csv saved.
Group:  2020-09-26.csv saved.
Group:  2020-09-27.csv saved.
Group:  2020-09-28.csv saved.
Group:  2020-09-29.csv saved.
Group:  2020-09-30.csv saved.
Group:  2020-10-01.csv saved.
Group:  2020-10-02.csv saved.
Group:  2020-10-03.csv saved.
Group:  2020-10-04.csv saved.
Group:  2020-10-05.csv saved.
Group:  2020-10-06.csv saved.
Group:  2020-10-07.csv saved.
Group:  2020-10-08.csv saved.
Group:  2020-10-09.csv saved.
Group:  2020-10-10.csv saved.
Group:  2020-10-11.csv saved.
Group:  2020-10-12.csv saved.
Group:  2020-10-13.csv saved.
Group:  2020-10-14.csv saved.
Group:  2020-10-15.csv saved.
Group:  2020-10-16.csv saved.
Group:  2020-10-17.csv saved.
Group:  2020-10-18.csv saved.
Group:  2020-10-19.csv saved.
Group:  2020-10-20.csv saved.
Group:  2020-10-21.csv saved.
Group:  2020-10-22.csv saved.
Group:  2020-10-23.csv saved.
Group:  20